In [1]:
import tensorflow as tf
import iris_data

args = {'batch_size': 100, 'train_steps': 2000}

In [2]:
def my_model(features, labels, mode, params):
    """DNN with three hidden layers, and dropout of 0.1 probability."""
    # Create three fully connected layers each layer having a dropout
    # probability of 0.1.
    net = tf.feature_column.input_layer(features, params['feature_columns'])
    for units in params['hidden_units']:
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # Compute logits (1 per class).
    logits = tf.layers.dense(net, params['n_classes'], activation=None)

    # Compute predictions.
    predicted_classes = tf.argmax(logits, 1)
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            'class_ids': predicted_classes[:, tf.newaxis],
            'probabilities': tf.nn.softmax(logits),
            'logits': logits,
        }
        return tf.estimator.EstimatorSpec(mode, predictions=predictions)

    # Compute loss.
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # Compute evaluation metrics.
    accuracy = tf.metrics.accuracy(labels=labels,
                                   predictions=predicted_classes,
                                   name='acc_op')
    metrics = {'accuracy': accuracy}
    tf.summary.scalar('accuracy', accuracy[1])

    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(
            mode, loss=loss, eval_metric_ops=metrics)

    # Create training op.
    assert mode == tf.estimator.ModeKeys.TRAIN

    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [3]:
# Fetch data paths
train_path, test_path = iris_data.maybe_download()

In [4]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth',
                    'PetalLength', 'PetalWidth']
# Feature columns describe how to use the input.
my_feature_columns = []
for key in CSV_COLUMN_NAMES:
    my_feature_columns.append(tf.feature_column.numeric_column(key=key))

In [5]:
my_checkpointing_config = tf.estimator.RunConfig(
    # save_checkpoints_steps = 250, # Save checkpoints every 250 steps
    save_checkpoints_secs = 1,  # Save checkpoints every 20 minutes.
    keep_checkpoint_max = 10,       # Retain the 10 most recent checkpoints.
)
# Build 2 hidden layer DNN with 10, 10 units respectively.
classifier = tf.estimator.Estimator(
    model_fn=my_model,
    model_dir="models/iris_custom",
    params={
        'feature_columns': my_feature_columns,
        # Two hidden layers of 10 nodes each.
        'hidden_units': [10, 10],
        # The model must choose between 3 classes.
        'n_classes': 3,
    },
    # Checkpoint config
    config=my_checkpointing_config)

INFO:tensorflow:Using config: {'_model_dir': 'models/iris_custom', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 1, '_session_config': None, '_keep_checkpoint_max': 10, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f9902e91d68>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [6]:
# Train the Model.
classifier.train(
    input_fn=lambda:iris_data.csv_input_fn(train_path, args["batch_size"]),
    steps=args["train_steps"])

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into models/iris_custom/model.ckpt.
INFO:tensorflow:loss = 1.3081301, step = 0
INFO:tensorflow:global_step/sec: 230.365
INFO:tensorflow:loss = 0.14116025, step = 100 (0.435 sec)
INFO:tensorflow:global_step/sec: 268.316
INFO:tensorflow:loss = 0.04915283, step = 200 (0.372 sec)
INFO:tensorflow:Saving checkpoints for 229 into models/iris_custom/model.ckpt.
INFO:tensorflow:global_step/sec: 240.315
INFO:tensorflow:loss = 0.04774713, step = 300 (0.416 sec)
INFO:tensorflow:global_step/sec: 263.503
INFO:tensorflow:loss = 0.1273242, step = 400 (0.380 sec)
INFO:tensorflow:Saving checkpoints for 485 into models/iris_custom/model.ckpt.
INFO:tensorflow:global_step/sec: 237.608
INFO:tensorflow:loss = 0.07905531, step = 500 (0.

In [7]:
# Evaluate the model.
eval_result = classifier.evaluate(
    input_fn=lambda:iris_data.csv_eval_input_fn(test_path, args["batch_size"]))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2018-07-09-11:50:20
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris_custom/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2018-07-09-11:50:21
INFO:tensorflow:Saving dict for global step 2000: accuracy = 0.96666664, global_step = 2000, loss = 0.053501632

Test set accuracy: 0.967



In [8]:
# Generate predictions from the model
expected = ['Setosa', 'Versicolor', 'Virginica']
predict_x = {
    'SepalLength': [5.1, 5.9, 6.9],
    'SepalWidth': [3.3, 3.0, 3.1],
    'PetalLength': [1.7, 4.2, 5.4],
    'PetalWidth': [0.5, 1.5, 2.1],
}

predictions = classifier.predict(
    input_fn=lambda:iris_data.eval_input_fn(predict_x,
                                            labels=None,
                                            batch_size=args['batch_size']))

for pred_dict, expec in zip(predictions, expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')

    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print(template.format(iris_data.SPECIES[class_id],
                          100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from models/iris_custom/model.ckpt-2000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "Setosa" (99.9%), expected "Setosa"

Prediction is "Versicolor" (99.9%), expected "Versicolor"

Prediction is "Virginica" (99.5%), expected "Virginica"
